In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from geopy.extra.rate_limiter import RateLimiter
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [3]:
houses = "https://www.domain.com.au/sale/eastern-suburbs-vic/?sort=price-asc&amp;page=1"
response = get(houses, headers=headers)

In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
house_containers = html_soup.find_all('div', class_="css-1gkcyyc")

In [6]:
prices = []
addresses = []
suburbs = []
states = []
postcodes = []
bedss = []
bathrooms = []
parkings = []
types = []
urls = []

In [7]:
%%time
n_pages = 0
for page in range(1,50):
    n_pages += 1
    houses_url = 'https://www.domain.com.au/sale/eastern-suburbs-vic/?ptype=duplex,free-standing,new-home-designs,new-house-land,semi-detached,terrace,town-house,villa&price=250000-750000&sort=price-asc'+ '&page='+str(page)
    r = get(houses_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="css-1gkcyyc")
    if house_containers != []:
        for container in house_containers:            
             #Price
            price = container.find_all('p')[0].text[0:9]
            price = price.replace(',','')
            price = price.replace('$','')
            price = price.replace(' ','')
            prices.append(price)
            #Address
            address = container.find_all('span')[0].text
            address = address.replace(',\xa0','')
            addresses.append(address)
            #Suburb
            suburb = container.find_all('span')[2].text
            suburbs.append(suburb)
            #State
            state = container.find_all('span')[3].text
            states.append(state)
            #Postcode
            postcode = container.find_all('span')[4].text
            postcodes.append(postcode)
            #Beds
            beds = container.find_all('span')[5].text
            beds = beds.replace(' Beds', '')
            beds = beds.replace(' Bed', '')
            beds = beds.replace('âˆ’', '')
            bedss.append(beds)
            
            #Bathrooms
            bathroom = container.find_all('span')[8].text
            bathroom = bathroom.replace(' Bath', '')
            bathroom = bathroom.replace('s', '')
            bathroom = bathroom.replace('âˆ’', '')
            bathrooms.append(bathroom)
            
            #Parking Spots
            parking = container.find_all('span')[11].text
            parking = parking.replace(' Parking', '')
            parking = parking.replace('âˆ’', '')
            parkings.append(parking)
            #Url
            url = container.find_all('a')[0].get('href')
            urls.append(url)
            #Type
#            type = container.find_all('span')[14].text
 #           types.append(type)
            
    else:
        break
        
    #sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(addresses)))
print(houses_url)

You scraped 33 pages containing 563 properties.
https://www.domain.com.au/sale/eastern-suburbs-vic/?ptype=duplex,free-standing,new-home-designs,new-house-land,semi-detached,terrace,town-house,villa&price=250000-750000&sort=price-asc&page=33
Wall time: 15.7 s


In [8]:
cols = ['Price', 'Address', 'Suburb', 'State', 'Postcode', 'Beds', 'Bathrooms', 'Parking_Spots', 'URL']
df = pd.DataFrame({'Price':prices, 
                   'Address': addresses, 
                   'Suburb': suburbs, 
                   'State': states, 
                   'Postcode': postcodes, 
                   'Beds': bedss, 
                   'Bathrooms': bathrooms, 
                   'Parking_Spots': parkings, 'URL': urls})
df = df.drop_duplicates(subset='Address', keep='first')
#df.head(50)

In [9]:
new_df = df[df.Price.apply(lambda x: x.isnumeric())]
#new_df.head(50)
new_df.shape

(385, 9)

In [10]:
df1 = new_df[df.State != 'VIC']
df1

<ipython-input-10-80e93b9d38ec>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = new_df[df.State != 'VIC']


,Price,Address,Suburb,State,Postcode,Beds,Bathrooms,Parking_Spots,URL
29,475000,YARRA JUNCTION VIC 3797,VIC,3797,3 Beds,3,1,2,https://www.domain.com.au/yarra-junction-vic-3...
91,549950,HEATHMONT VIC 3135,VIC,3135,2 Beds,2,1,1,https://www.domain.com.au/heathmont-vic-3135-2...
154,590000,HEALESVILLE VIC 3777,VIC,3777,4 Beds,4,2,2,https://www.domain.com.au/healesville-vic-3777...
171,599950,YARRA GLEN VIC 3775,VIC,3775,4 Beds,4,2,2,https://www.domain.com.au/yarra-glen-vic-3775-...
423,699000,RINGWOOD VIC 3134,VIC,3134,2 Beds,2,2,1,https://www.domain.com.au/ringwood-vic-3134-20...
521,749950,BORONIA VIC 3155,VIC,3155,3 Beds,3,2,2,https://www.domain.com.au/boronia-vic-3155-201...


In [11]:
new_df = new_df[new_df.State == 'VIC']
new_df.shape

(379, 9)

In [12]:
new_df = new_df[new_df.Parking_Spots <= '5']
new_df.shape

(347, 9)

In [13]:
#new_df.head(50)

In [14]:
#new_df.to_csv('testinghousesdata.csv', index=False)

In [15]:
new_df['Full_Address'] = new_df['Address'] +' '+ new_df['Suburb'] + ' ' + new_df['Postcode'] + ' ' + new_df['State']

In [16]:
new_df

,Price,Address,Suburb,State,Postcode,Beds,Bathrooms,Parking_Spots,URL,Full_Address
0,308750,20/74 Warrandyte,RINGWOOD,VIC,3134,1,1,1,https://www.domain.com.au/20-74-warrandyte-rin...,20/74 Warrandyte RINGWOOD 3134 VIC
1,684500,4/21 English Street,SEVILLE,VIC,3139,4,2,2,https://www.domain.com.au/4-21-english-street-...,4/21 English Street SEVILLE 3139 VIC
4,340000,50 Hilltop Court,YARRA JUNCTION,VIC,3797,2,1,1,https://www.domain.com.au/50-hilltop-court-yar...,50 Hilltop Court YARRA JUNCTION 3797 VIC
5,365000,211/31-43 Victoria Street,DONCASTER,VIC,3108,1,1,1,https://www.domain.com.au/211-31-43-victoria-s...,211/31-43 Victoria Street DONCASTER 3108 VIC
6,385000,59/330 Springvale Road,DONVALE,VIC,3111,2,1,1,https://www.domain.com.au/59-330-springvale-ro...,59/330 Springvale Road DONVALE 3111 VIC
...,...,...,...,...,...,...,...,...,...,...
546,750000,152 Manchester Road,MOOROOLBARK,VIC,3138,3,2,2,https://www.domain.com.au/152-manchester-road-...,152 Manchester Road MOOROOLBARK 3138 VIC
547,750000,35 Kirkford Drive,MOOROOLBARK,VIC,3138,4,2,2,https://www.domain.com.au/35-kirkford-drive-mo...,35 Kirkford Drive MOOROOLBARK 3138 VIC
550,750000,72 Maidstone Street,RINGWOOD,VIC,3134,4,2,3,https://www.domain.com.au/72-maidstone-street-...,72 Maidstone Street RINGWOOD 3134 VIC
556,750000,39 Joelson Avenue,SCORESBY,VIC,3179,3,2,2,https://www.domain.com.au/39-joelson-avenue-sc...,39 Joelson Avenue SCORESBY 3179 VIC


In [17]:
geolocator = Nominatim(user_agent="my-app")

In [18]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [19]:
new_df['city_coord']  = new_df['Full_Address'].apply(geocode)

In [20]:
new_df

,Price,Address,Suburb,State,Postcode,Beds,Bathrooms,Parking_Spots,URL,Full_Address,city_coord
0,308750,20/74 Warrandyte,RINGWOOD,VIC,3134,1,1,1,https://www.domain.com.au/20-74-warrandyte-rin...,20/74 Warrandyte RINGWOOD 3134 VIC,"(Warrandyte Road, Ringwood, City of Maroondah,..."
1,684500,4/21 English Street,SEVILLE,VIC,3139,4,2,2,https://www.domain.com.au/4-21-english-street-...,4/21 English Street SEVILLE 3139 VIC,"(English Street, Seville, Shire of Yarra Range..."
4,340000,50 Hilltop Court,YARRA JUNCTION,VIC,3797,2,1,1,https://www.domain.com.au/50-hilltop-court-yar...,50 Hilltop Court YARRA JUNCTION 3797 VIC,"(Hilltop Court, Yarra Junction, Shire of Yarra..."
5,365000,211/31-43 Victoria Street,DONCASTER,VIC,3108,1,1,1,https://www.domain.com.au/211-31-43-victoria-s...,211/31-43 Victoria Street DONCASTER 3108 VIC,None
6,385000,59/330 Springvale Road,DONVALE,VIC,3111,2,1,1,https://www.domain.com.au/59-330-springvale-ro...,59/330 Springvale Road DONVALE 3111 VIC,None
...,...,...,...,...,...,...,...,...,...,...,...
546,750000,152 Manchester Road,MOOROOLBARK,VIC,3138,3,2,2,https://www.domain.com.au/152-manchester-road-...,152 Manchester Road MOOROOLBARK 3138 VIC,"(Manchester Road, Manchester, Mooroolbark, Mon..."
547,750000,35 Kirkford Drive,MOOROOLBARK,VIC,3138,4,2,2,https://www.domain.com.au/35-kirkford-drive-mo...,35 Kirkford Drive MOOROOLBARK 3138 VIC,"(Kirkford Drive, Rolling Hills, Mooroolbark, S..."
550,750000,72 Maidstone Street,RINGWOOD,VIC,3134,4,2,3,https://www.domain.com.au/72-maidstone-street-...,72 Maidstone Street RINGWOOD 3134 VIC,"(72, Maidstone Street, Ringwood, City of Maroo..."
556,750000,39 Joelson Avenue,SCORESBY,VIC,3179,3,2,2,https://www.domain.com.au/39-joelson-avenue-sc...,39 Joelson Avenue SCORESBY 3179 VIC,"(Joelson Avenue, Scoresby, City of Knox, Victo..."


In [21]:
#newtest_df = new_df.head(20)

In [22]:
#newtest_df

In [23]:
#newtest_df['city_coord']  = newtest_df['Full_Address'].apply(geolocator.geocode)

In [24]:
new_df = new_df.dropna(how='any',axis=0) ## removes any rows geocode couldnt discover 

In [25]:
new_df['Latitude'] = new_df['city_coord'].apply(lambda x: (x.latitude))
new_df['Longitude'] = new_df['city_coord'].apply(lambda x: (x.longitude))

<ipython-input-25-01a93447ed14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Latitude'] = new_df['city_coord'].apply(lambda x: (x.latitude))
<ipython-input-25-01a93447ed14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Longitude'] = new_df['city_coord'].apply(lambda x: (x.longitude))


In [26]:
new_df

,Price,Address,Suburb,State,Postcode,Beds,Bathrooms,Parking_Spots,URL,Full_Address,city_coord,Latitude,Longitude
0,308750,20/74 Warrandyte,RINGWOOD,VIC,3134,1,1,1,https://www.domain.com.au/20-74-warrandyte-rin...,20/74 Warrandyte RINGWOOD 3134 VIC,"(Warrandyte Road, Ringwood, City of Maroondah,...",-37.808028,145.230474
1,684500,4/21 English Street,SEVILLE,VIC,3139,4,2,2,https://www.domain.com.au/4-21-english-street-...,4/21 English Street SEVILLE 3139 VIC,"(English Street, Seville, Shire of Yarra Range...",-37.769644,145.463167
4,340000,50 Hilltop Court,YARRA JUNCTION,VIC,3797,2,1,1,https://www.domain.com.au/50-hilltop-court-yar...,50 Hilltop Court YARRA JUNCTION 3797 VIC,"(Hilltop Court, Yarra Junction, Shire of Yarra...",-37.786205,145.613378
10,380000,12 Ronald Grove,MILLGROVE,VIC,3799,2,1,1,https://www.domain.com.au/12-ronald-grove-mill...,12 Ronald Grove MILLGROVE 3799 VIC,"(Ronald Grove, Millgrove, Warburton, Shire of ...",-37.751510,145.660950
11,400000,2 James Street,MILLGROVE,VIC,3799,3,1,2,https://www.domain.com.au/2-james-street-millg...,2 James Street MILLGROVE 3799 VIC,"(James Street, Millgrove, Warburton, Shire of ...",-37.747870,145.660990
...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,750000,152 Manchester Road,MOOROOLBARK,VIC,3138,3,2,2,https://www.domain.com.au/152-manchester-road-...,152 Manchester Road MOOROOLBARK 3138 VIC,"(Manchester Road, Manchester, Mooroolbark, Mon...",-37.778667,145.312497
547,750000,35 Kirkford Drive,MOOROOLBARK,VIC,3138,4,2,2,https://www.domain.com.au/35-kirkford-drive-mo...,35 Kirkford Drive MOOROOLBARK 3138 VIC,"(Kirkford Drive, Rolling Hills, Mooroolbark, S...",-37.759848,145.318538
550,750000,72 Maidstone Street,RINGWOOD,VIC,3134,4,2,3,https://www.domain.com.au/72-maidstone-street-...,72 Maidstone Street RINGWOOD 3134 VIC,"(72, Maidstone Street, Ringwood, City of Maroo...",-37.826084,145.224739
556,750000,39 Joelson Avenue,SCORESBY,VIC,3179,3,2,2,https://www.domain.com.au/39-joelson-avenue-sc...,39 Joelson Avenue SCORESBY 3179 VIC,"(Joelson Avenue, Scoresby, City of Knox, Victo...",-37.888003,145.234437


In [27]:
melbourne = 'Melbourne, City of Melbourne'
location = geolocator.geocode(melbourne)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne is -37.80424240000001, 144.94112315764534.


In [28]:
# create map of melbourne using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city_coord, house_url in zip(new_df['Latitude'], new_df['Longitude'], new_df['Full_Address'], new_df['URL']):
    label = '{}, {}'.format(city_coord, house_url)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [29]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'QDYQFI0MNMB4UZE4DTYWXUTOG41KQAXCCZXPCZ1FERD5MCB3' # Foursquare ID
CLIENT_SECRET = 'WSUDLMVMNZMTDGBPS05EDGUBFY2RY5X5NL5UMCK303RZNOFO' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QDYQFI0MNMB4UZE4DTYWXUTOG41KQAXCCZXPCZ1FERD5MCB3
CLIENT_SECRET:WSUDLMVMNZMTDGBPS05EDGUBFY2RY5X5NL5UMCK303RZNOFO


In [30]:

def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(           
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Address', 
                  'Address Latitude', 
                  'Address Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
location_venues = getNearbyVenues(names=new_df['Full_Address'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )

20/74 Warrandyte RINGWOOD 3134 VIC
4/21 English Street SEVILLE 3139 VIC
50 Hilltop Court YARRA JUNCTION 3797 VIC
12 Ronald Grove MILLGROVE 3799 VIC
2 James Street MILLGROVE 3799 VIC
11 Christine Street MILLGROVE 3799 VIC
20 Christine Street MILLGROVE 3799 VIC
61/330 Springvale Road DONVALE 3111 VIC
24 Croydon Road CROYDON 3136 VIC
38/330 Springvale Road DONVALE 3111 VIC
5 Bacchus Crescent MILLGROVE 3799 VIC
16 Clarke Avenue WARBURTON 3799 VIC
34 McKenzie King Drive MILLGROVE 3799 VIC
37 Woods Point Road WARBURTON 3799 VIC
14 Alan Grove WOORI YALLOCK 3139 VIC
5 Eliza Mews LILYDALE 3140 VIC
4/5 Cypress Avenue BORONIA 3155 VIC
2/62 Croydondale Drive MOOROOLBARK 3138 VIC
5 Toby Place MOOROOLBARK 3138 VIC
3/62 Croydondale Drive MOOROOLBARK 3138 VIC
13 Greenwood Drive LAUNCHING PLACE 3139 VIC
2/35 Little yarra rd YARRA JUNCTION 3797 VIC
5/38 The Eyrie LILYDALE 3140 VIC
3/38 The Eyrie LILYDALE 3140 VIC
2/80 Castella Street LILYDALE 3140 VIC
2/7 Stamford Crescent ROWVILLE 3178 VIC
1 Palm Place

In [32]:
location_venues.head(10)

,Address,Address Latitude,Address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Hoyts Cinemas,-37.812432,145.230246,Movie Theater
1,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,The Pancake Parlour,-37.812524,145.230228,Restaurant
2,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Kyoto Cafe,-37.814450,145.228856,Japanese Restaurant
3,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Eastland Shopping Centre,-37.813471,145.229031,Shopping Mall
4,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Costco,-37.814539,145.225573,Warehouse Store
5,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Stirr it Up Cafe,-37.812780,145.234430,Café
6,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Starbucks,-37.812428,145.230088,Coffee Shop
7,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Paco's Taco's,-37.814618,145.228810,Mexican Restaurant
8,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Officeworks,-37.813785,145.232098,Paper / Office Supplies Store
9,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,JB Hi-Fi,-37.813381,145.228325,Electronics Store


In [112]:
my_venues = location_venues[(location_venues['Venue Category'] == 'Park') | 
                         (location_venues['Venue Category'] == 'Supermarket') |
                         (location_venues['Venue Category'] == 'Gym') | 
#                         (location_venues['Venue Category'] == 'Train Station') | 
                         (location_venues['Venue Category'] == 'Café')]



In [117]:
my_venues
#my_countedvenues = my_venues.groupby('Street').count()
#my_venues.groupby('Street').count()

,Address,Address Latitude,Address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Stirr it Up Cafe,-37.812780,145.234430,Café
12,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Coles,-37.812432,145.230085,Supermarket
16,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,The Firehouse,-37.813557,145.233079,Café
19,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Woolworths,-37.812519,145.228582,Supermarket
35,20/74 Warrandyte RINGWOOD 3134 VIC,-37.808028,145.230474,Mullum Reserve,-37.803442,145.237954,Park
...,...,...,...,...,...,...,...
2844,39 Joelson Avenue SCORESBY 3179 VIC,-37.888003,145.234437,Degani Bakery Cafe,-37.882127,145.233900,Café
2847,39 Joelson Avenue SCORESBY 3179 VIC,-37.888003,145.234437,Knox Gardens Reserve,-37.880510,145.229047,Park
2852,10 Frances Avenue YARRA GLEN 3775 VIC,-37.651635,145.382260,McKenzie Reserve,-37.656327,145.374129,Park
2853,10 Frances Avenue YARRA GLEN 3775 VIC,-37.651635,145.382260,Marsala Cafe,-37.657410,145.375027,Café


In [35]:
#my_countedvenues

In [36]:
#my_venues[my_venues['Venue Category'].isin(['Supermarket', 'Park', 'Gym'])]

In [84]:
park_venues = location_venues[location_venues['Venue Category'] == 'Park']
park_venues = park_venues.drop_duplicates(subset='Address', keep='first')

In [85]:
supermarket_venues = (location_venues[location_venues['Venue Category'] == 'Supermarket'])
supermarket_venues = supermarket_venues.drop_duplicates(subset='Address', keep='first')

In [86]:
#station_venues = (location_venues[location_venues['Venue Category'] == 'Train Station'])
#station_venues = station_venues.drop_duplicates(subset='Address', keep='first')

In [87]:
gym_venues = (location_venues[location_venues['Venue Category'] == 'Gym'])
gym_venues = gym_venues.drop_duplicates(subset='Address', keep='first')

In [88]:
cafe_venues = (location_venues[location_venues['Venue Category'] == 'Café'])
cafe_venues = cafe_venues.drop_duplicates(subset='Address', keep='first')

In [89]:
my_venues = park_venues.append(supermarket_venues)

In [90]:
my_venues = my_venues.append(gym_venues)

In [91]:
my_venues = my_venues.append(cafe_venues)

In [ ]:
#code to remove all duplicate addresses, will use to map addresses again on map
#my_venues = my_venues.drop_duplicates(subset='Street', keep=False)
#code to show venues for an address

In [94]:
my_countedvenues = my_venues.groupby('Address').count()

In [110]:
my_countedvenues = my_countedvenues[my_countedvenues['Venue Category'] == 4]
my_countedvenues = my_countedvenues.drop(columns=['Address Latitude', 'Address Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude'])

In [111]:
my_countedvenues

,Venue Category
Address,
2/95 Scoresby Road BAYSWATER 3153 VIC,4
5/9 Citrus Court DONCASTER 3108 VIC,4


In [183]:
#compression_opts = dict(method='zip',
#                        archive_name='available_housesTEST.csv')  
#new_df.to_csv('available_housesTEST.zip', index=False,
#          compression=compression_opts)  

In [31]:
new_df.to_csv('testinghousesdata.csv', index=False)

In [80]:
location_venues.to_csv('housevenueswithin500M.csv', index=False)

In [344]:
my_countedvenues.to_csv('venuetest.csv', index=False)